# Lab 3 — clustering

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle
import sklearn.cluster as cl

# Import bokeh
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, ResetTool, PanTool, WheelZoomTool, SaveTool
output_notebook()
# Categorial colors
from bokeh.palettes import Dark2_8

%matplotlib inline
plt.style.use("ggplot")

Loading BokehJS ...

## 3.11

In [2]:
with open("arrrrr.pickle", "rb") as file:
    tagsPCA_p = pickle.load(file, encoding="utf-8")

In [3]:
tagsPCA = list(tagsPCA_p.values())
ks = [2, 3, 4, 5]

In [4]:
def k_means(k, data):
    kmeans = cl.KMeans(k)
    kmeans.fit(data)
    print(kmeans)
    return kmeans

In [5]:
def create_cluster(kmeans, names, dims):
    source = ColumnDataSource(
        data={
            "x": [x[dims[0]] for x in tagsPCA],
            "y": [x[dims[1]] for x in tagsPCA],
            "name": [x for x in names],
            "color": [Dark2_8[x] for x in kmeans.labels_],
        })

    hover = HoverTool(
        tooltips=[
            ("Name", "@name"),
        ])
    tools = [hover, ResetTool(), PanTool(), WheelZoomTool(), SaveTool()]
    p = figure(plot_width=960, plot_height=360, tools=tools, title="Mouse over the dots")
    p.circle("x", "y", source=source, size=20, color="color", alpha=0.5)
    #show(p, notebook_handle=True)
    return p

In [6]:
def visualize_cluster(cluster_plot):
    show(cluster_plot, notebook_handle=True)

In [7]:
plots = []

for k in ks:
    print(k)
    plots.append(create_cluster(k_means(k, tagsPCA), tagsPCA_p.keys(), (0, 1)))

2
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
3
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
4
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
5
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


In [8]:
# show(vplot(plots))

# gridplot is unfortunately not available on the cluster
# show(gridplot(plots))

In [9]:
visualize_cluster(plots[0])

In [10]:
visualize_cluster(plots[1])

In [11]:
visualize_cluster(plots[2])

In [12]:
visualize_cluster(plots[3])

#### Which principal directions separate the clusters well?
>

## 3.12

In [ ]:
# Data Frame of all movies and genres. Taken directly from the introduction in dimred.
data_movies = sc.textFile("./../").map(json.loads)

# Frame of genres in movie order.
intermediateGenresDF = pd.DataFrame(data_movies.map(itemgetter("genres")).collect())
# Frame of all unique genres. There are 20 genres, including "No genre listed", once we drop None.
genresDF = pd.DataFrame(pd.DataFrame(intermediateGenresDF.values.flatten()).drop_duplicates().reset_index()[0])
# We drop "None", as it is to disregard.
genresDF = pd.DataFrame(genresDF[~genresDF[0].isnull()].reset_index()[0])

# We need anothe dataframe that lists all the genre lsits with the right movieId as they are NOT continuous
genresListsDF = pd.DataFrame(data_movies.map(itemgetter("genres", "movieId")).collect(),columns=["genres", "movieId"])

In [53]:
def J_distance(a, b):
    set1 = set(a)
    set2 = set(b)
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return 1 - ( float(len(intersection)) / len(union) )

In [54]:
def k_medioids(k, points, threshold, maxsteps=1500):
    
    shuffle = np.random.randint(len(points), size=k)
    medioids = points[shuffle]
    
    for step in range(maxsteps):
        dists = np.zeros((len(points), k))
        #For each medioid, the distance between each point and this medioids.
        for i in range(len(points)):
            for j in range(k):
                dists[i,j] = J_distance(points[i], medioids[j])
                
        labels = np.argpartition(dists,1,1)[:,0]
        
        medioids2 = np.zeros(medioids.shape)
        
        for i in range(k):
            clusters = points[labels == i]
            real_index = np.argwhere(closest == i)
            sums = np.zeros(len(clusters))
            for j, x in enumerate(clusters):
                for y in clusters:
                    sums[j] += J_distance(x, y)
            medioids2[i] = clusters[np.argmin(sums)]

        diff = medioids2 - medioids
        dist = np.max(np.linalg.norm(diff,axis=0))
        
        if(dist < thershold):
            return labels
        medioids = medioids2
    return labels

In [55]:
with open("most-rated.pickle", "rb") as f:
    movies = pk.load(f, encoding="utf-8")
data = pd.DataFrame(list(movies), columns=['movieId', 'title'])

NameError: name 'pk' is not defined

In [ ]:
clusterKmed = k_mediods(2, data, 0.00005)